# dataset

In [1]:
# from TTS.tts.layers.xtts.tokenizer import multilingual_cleaners
import gc
from tqdm import tqdm
import torch
import torchaudio
import pandas
from faster_whisper import WhisperModel
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
def format_audio_list(audio_files, target_language="en", out_path=None, buffer=0.2, eval_percentage=0.15, speaker_name="coqui", gradio_progress=None):
    audio_total_size = 0
    # make sure that ooutput file exists
    os.makedirs(out_path, exist_ok=True)

    # print lenght of audio files
    print(f"Found {len(audio_files)} audio files!")

    # Loading Whisper
    device = "cuda" if torch.cuda.is_available() else "cpu" 
    # device = "cpu"
    print(f"Using {device} device")

    print("Loading Whisper Model!")
    asr_model = WhisperModel("large-v2", device=device, compute_type="float32")

    metadata = {"audio_file": [], "text": [], "speaker_name": []}

    if gradio_progress is not None:
        tqdm_object = gradio_progress.tqdm(audio_files, desc="Formatting...")
    else:
        tqdm_object = tqdm(audio_files)

    for audio_path in tqdm_object:
        wav, sr = torchaudio.load(audio_path)
        # stereo to mono if needed
        if wav.size(0) != 1:
            wav = torch.mean(wav, dim=0, keepdim=True)

        wav = wav.squeeze()
        audio_total_size += (wav.size(-1) / sr)

        print(f"Transcribing {audio_path}...")
        segments, _ = asr_model.transcribe(audio_path, word_timestamps=True, language=target_language,)
        segments = list(segments)
        print(f"Found {len(segments)} segments!")
        i = 0
        sentence = ""
        sentence_start = None
        first_word = True
        # added all segments words in a unique list
        words_list = []
        for _, segment in enumerate(segments):
            words = list(segment.words)
            words_list.extend(words)

        # process each word
        for word_idx, word in enumerate(words_list):
            if first_word:
                sentence_start = word.start
                # If it is the first sentence, add buffer or get the begining of the file
                if word_idx == 0:
                    sentence_start = max(sentence_start - buffer, 0)  # Add buffer to the sentence start
                else:
                    # get previous sentence end
                    previous_word_end = words_list[word_idx - 1].end
                    # add buffer or get the silence midle between the previous sentence and the current one
                    sentence_start = max(sentence_start - buffer, (previous_word_end + sentence_start)/2)

                sentence = word.word
                first_word = False
            else:
                sentence += word.word

            if word.word[-1] in ["!", ".", "?"]:
                sentence = sentence[1:]
                # Expand number and abbreviations plus normalization
                # sentence = multilingual_cleaners(sentence, target_language)
                audio_file_name, _ = os.path.splitext(os.path.basename(audio_path))

                audio_file = f"wavs/{audio_file_name}_{str(i).zfill(8)}.wav"

                # Check for the next word's existence
                if word_idx + 1 < len(words_list):
                    next_word_start = words_list[word_idx + 1].start
                else:
                    # If don't have more words it means that it is the last sentence then use the audio len as next word start
                    next_word_start = (wav.shape[0] - 1) / sr

                # Average the current word end and next word start
                word_end = min((word.end + next_word_start) / 2, word.end + buffer)
                
                absoulte_path = os.path.join(out_path, audio_file)
                os.makedirs(os.path.dirname(absoulte_path), exist_ok=True)
                i += 1
                first_word = True

                audio = wav[int(sr*sentence_start):int(sr*word_end)].unsqueeze(0)
                # if the audio is too short ignore it (i.e < 0.33 seconds)
                if audio.size(-1) >= sr/3:
                    torchaudio.save(absoulte_path,
                        audio,
                        sr
                    )
                else:
                    continue

                metadata["audio_file"].append(audio_file)
                metadata["text"].append(sentence)
                metadata["speaker_name"].append(speaker_name)

    df = pandas.DataFrame(metadata)
    df = df.sample(frac=1)
    num_val_samples = int(len(df)*eval_percentage)

    df_eval = df[:num_val_samples]
    df_train = df[num_val_samples:]

    df_train = df_train.sort_values('audio_file')
    train_metadata_path = os.path.join(out_path, "metadata_train.csv")
    df_train.to_csv(train_metadata_path, sep="|", index=False)

    eval_metadata_path = os.path.join(out_path, "metadata_eval.csv")
    df_eval = df_eval.sort_values('audio_file')
    df_eval.to_csv(eval_metadata_path, sep="|", index=False)

    # deallocate VRAM and RAM
    del asr_model, df_train, df_eval, df, metadata
    gc.collect()

    return train_metadata_path, eval_metadata_path, audio_total_size

def clear_gpu_cache():
    # clear the GPU cache

    if torch.cuda.is_available():
        print("Clearing GPU Cache...")
        torch.cuda.empty_cache()

c:\Users\tibed\anaconda3\envs\Research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
audio_path = "audio\Alan_Wake_Voice_Short.wav"

In [5]:

asr_model = WhisperModel("large-v3", device=device, compute_type="float16")

In [5]:
asr_model = WhisperModel("large-v2", device=device, compute_type="float16")
segments, _ = asr_model.transcribe(audio_path, word_timestamps=True, language="en")
segments = list(segments)
print(f"Found {len(segments)} segments!")
print(segments[0].words)

Found 75 segments!
[Word(start=0.0, end=0.14, word=' He', probability=0.062103271484375), Word(start=0.14, end=0.3, word=' was', probability=0.87890625), Word(start=0.3, end=0.62, word=' here,', probability=0.7958984375), Word(start=1.2, end=1.34, word=' in', probability=0.8759765625), Word(start=1.34, end=1.6, word=' Bright', probability=0.85888671875), Word(start=1.6, end=2.0, word=' Falls.', probability=0.6923828125)]


In [9]:
segments, _ = asr_model.transcribe(audio_path, word_timestamps=True, language="en")
segments = list(segments)
print(f"Found {len(segments)} segments!")
print(segments[0].words)

Found 39 segments!
[Word(start=0.0, end=0.1, word=' He', probability=0.9794921875), Word(start=0.1, end=0.2, word=' was', probability=1.0), Word(start=0.2, end=0.48, word=' here.', probability=1.0), Word(start=0.7, end=1.24, word=' In', probability=0.9892578125), Word(start=1.24, end=1.46, word=' Bright', probability=0.99072265625), Word(start=1.46, end=1.76, word=' Falls.', probability=0.99560546875), Word(start=2.24, end=2.8, word=' I', probability=0.9990234375), Word(start=2.8, end=2.94, word=' could', probability=1.0), Word(start=2.94, end=3.24, word=' feel', probability=1.0), Word(start=3.24, end=3.52, word=' him', probability=1.0), Word(start=3.52, end=3.62, word=' as', probability=1.0), Word(start=3.62, end=3.74, word=' a', probability=1.0), Word(start=3.74, end=4.02, word=' growing', probability=1.0), Word(start=4.02, end=4.44, word=' pressure', probability=1.0), Word(start=4.44, end=4.8, word=' in', probability=0.99951171875), Word(start=4.8, end=4.9, word=' my', probability=1

In [7]:
import gc
import torch
import torchaudio
import pandas
from faster_whisper import WhisperModel
import os
def create_dataset(output_path, audio_path, target_language="en", buffer=0.2, eval_percentage=0.15, speaker_name="coqui"):
    audio_files = [audio_path]

    os.makedirs(output_path, exist_ok=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    whisper_model = WhisperModel("large-v3", device=device, compute_type="float16")